In [2]:
import pickle
import numpy as np 
import matplotlib.pyplot as plt

In [3]:
with open("score_collector_en_de_small.pkl", "rb") as file:
    score_collector_en_de_small = pickle.load(file)

with open("score_collector_en_fr_small.pkl", "rb") as file:
    score_collector_en_fr_small = pickle.load(file)

with open("score_collector_en_ro_small.pkl", "rb") as file:
    score_collector_en_ro_small = pickle.load(file)

In [ ]:
with open("score_collector_en_de_base.pkl", "rb") as file:
    score_collector_en_de_base = pickle.load(file)

with open("score_collector_en_fr_small.pkl", "rb") as file:
    score_collector_en_fr_base = pickle.load(file)

with open("score_collector_en_ro_small.pkl", "rb") as file:
    score_collector_en_ro_base = pickle.load(file)